In [2]:
import pandas as pd
import os
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.styles import Alignment, Border, Side

def hyperlink_excel_file(output_filename):
    input_directory = 'C:\\RPA\\Input'  # 입력 디렉토리 경로
    target_directory = 'C:\\RPA\\Output'  # 저장할 디렉토리 경로
    
    xlsx_files = [f for f in os.listdir(input_directory) if f.lower().endswith('.xlsx')]
    
    if not xlsx_files:
        print("xlsx 파일이 없습니다.")
        return
    
    file_path = os.path.join(input_directory, xlsx_files[0])
    
    # xlsx 파일에서 데이터프레임 읽기
    df = pd.read_excel(file_path)

    # 워크북 불러오기
    wb = load_workbook(file_path)
    ws = wb.active

    # 테두리 스타일 정의
    thin_border = Border(left=Side(style='thin'), 
                         right=Side(style='thin'), 
                         top=Side(style='thin'), 
                         bottom=Side(style='thin'))

    # A~AH열의 너비 설정 및 자동 줄 바꿈, 테두리 적용
    for col in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'AA', 'AB', 'AC', 'AD']:
        ws.column_dimensions[col].width = 16
        
        for row in range(1, df.shape[0] + 2):  # 첫 번째 행은 헤더이므로 1을 더해준다.
            cell = ws[f"{col}{row}"]
            cell.alignment = Alignment(wrap_text=True)
            cell.border = thin_border  # 테두리 적용
    
    link_columns = ["본사상품링크", "고려기프트 상품링크", "네이버 쇼핑 링크", "공급사 상품링크"]
    for link_column in link_columns:
        for row_idx, link in enumerate(df[link_column], start=2):  # 첫 번째 행은 헤더이므로 인덱스 2부터 시작
            if pd.notna(link) and (link.startswith("http") or link.startswith("https")):  # 링크가 NaN이 아니며 http 또는 https로 시작할 때만 처리
                cell = ws.cell(row=row_idx, column=df.columns.get_loc(link_column) + 1)  # 열 인덱스 찾기
                cell.value = link  # 셀 값 설정
                cell.hyperlink = link  # 하이퍼링크 설정

    # 색 채우기 위한 PatternFill 객체 생성
    red_fill = PatternFill(start_color="FFFF00",
                           end_color="FFFF00",
                           fill_type="solid")

    for row_idx in range(2, df.shape[0] + 2):  # 첫 번째 행은 헤더이므로 인덱스 2부터 시작
        for col_name in ["가격차이(2)", "가격차이(3)"]:
            value = df.loc[row_idx - 2, col_name]  # Pandas DataFrame은 0-based indexing
            if pd.notna(value) and value < 0:
                for col in ws.iter_cols(min_row=row_idx, max_row=row_idx, min_col=1, max_col=ws.max_column):
                    for cell in col:
                        cell.fill = red_fill  # 셀 배경을 빨간색으로 설정
                break  # 행 전체에 색을 채웠으므로 다음 행으로 이동

    # 데이터프레임을 xlsx 파일로 저장
    output_file_path = os.path.join(target_directory, output_filename)
    wb.save(output_file_path)
    print(f"데이터가 {output_file_path}에 저장되었습니다.")


In [3]:
hyperlink_excel_file('Result20230927_150338.xlsx')

데이터가 C:\RPA\Output\Result20230927_150338.xlsx에 저장되었습니다.


In [3]:
import pandas as pd
import os
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Alignment, Border, Side
from urllib.parse import urlparse
from datetime import datetime

def filter_excel_file():
    input_directory = 'C:\\RPA\\Input'
    target_directory = 'C:\\RPA\\Output'
    
    xlsx_files = [f for f in os.listdir(input_directory) if f.lower().endswith('.xlsx')]
    
    if not xlsx_files:
        print("xlsx 파일이 없습니다.")
        return
    
    input_filename = xlsx_files[0].rsplit('.', 1)[0] # 확장자 제외한 파일명
    current_datetime = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_filename = f"{input_filename}_upload_{current_datetime}.xlsx"

    file_path = os.path.join(input_directory, xlsx_files[0])
    df = pd.read_excel(file_path)
    
    filtered_df = df[(df['가격차이(2)'] < 0) | (df['가격차이(3)'] < 0)]

    # 원하는 컬럼만 추출
    filtered_df = filtered_df[['구분', '담당자', '업체명', '업체코드', 'Code', '중분류카테고리', '상품명', '기본수량(1)', '판매단가(V포함)', '본사상품링크', '기본수량(2)', '판매단가(V포함)(2)', '가격차이(2)', '가격차이(2)(%)', '고려기프트 상품링크', '기본수량(3)', '판매단가(V포함)(3)', '가격차이(3)', '가격차이(3)(%)', '공급사명', '공급사 상품링크', '본사 이미지', '고려기프트 이미지', '네이버 이미지']]

    # 컬럼 이름 변경
    filtered_df.rename(columns={
        '구분': '구분(승인관리:A/가격관리:P)',
        '담당자': '담당자',
        '업체명': '공급사명',
        '업체코드': '공급처코드',
        'Code': '상품코드',
        '중분류카테고리': '카테고리(중분류)',
        '상품명': '상품명',
        '기본수량(1)': '본사 기본수량',
        '판매단가(V포함)': '판매단가1(VAT포함)',
        '본사상품링크': '본사링크',
        '기본수량(2)': '고려 기본수량',
        '판매단가(V포함)(2)': '판매단가2(VAT포함)',
        '가격차이(2)': '고려 가격차이',
        '가격차이(2)(%)': '고려 가격차이(%)',
        '고려기프트 상품링크': '고려 링크',
        '기본수량(3)': '네이버 기본수량',
        '판매단가(V포함)(3)': '판매단가3 (VAT포함)',
        '가격차이(3)': '네이버 가격차이',
        '가격차이(3)(%)': '네이버가격차이(%)',
        '공급사명': '네이버 공급사명',
        '공급사 상품링크': '네이버 링크',
        '본사 이미지': '해오름(이미지링크)',
        '고려기프트 이미지': '고려기프트(이미지링크)',
        '네이버 이미지': '네이버쇼핑(이미지링크)'
    }, inplace=True)

    output_path = os.path.join(target_directory, output_filename)
    filtered_df.to_excel(output_path, index=False)
    
    # 여기서부터 openpyxl을 사용해서 추가 작업
    wb = load_workbook(output_path)
    ws = wb.active
    
    # 스타일 설정
    thin_border = Border(left=Side(style='thin'), right=Side(style='thin'),
                         top=Side(style='thin'), bottom=Side(style='thin'))

    for row in ws.iter_rows(min_row=2, max_col=24, max_row=ws.max_row):
        for cell in row:
            cell.border = thin_border

    for col in ws.iter_cols(min_col=1, max_col=24):
        for cell in col:
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    
    # 너비와 높이 설정
    for col in ws.iter_cols(min_col=1, max_col=24, max_row=1):
        for cell in col:
            ws.column_dimensions[cell.column_letter].width = 5 # 15
            
    for row in ws.iter_rows(min_row=1, max_col=24, max_row=ws.max_row):
        for cell in row:
            ws.row_dimensions[cell.row].height = 16.5 # 100

    columns_to_check = [col[0].column for col in ws.iter_cols() if col[0].value in ['고려 기본수량', '판매단가2(VAT포함)', '고려 가격차이', '고려 가격차이(%)', '고려 링크', '네이버 기본수량', '판매단가3 (VAT포함)', '네이버 가격차이', '네이버가격차이(%)', '네이버 공급사명', '네이버 링크', '해오름(이미지링크)', '고려기프트(이미지링크)', '네이버쇼핑(이미지링크)']]
    
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
        for col_num in columns_to_check:
            cell = row[col_num-1]
            if cell.value == '-':
                cell.value = ''

    for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
        # "고려 링크"와 "네이버 링크" 체크
        goleo_link_cell = row[14]  # "고려 링크"는 15번째 열입니다.
        naver_link_cell = row[20]  # "네이버 링크"는 21번째 열입니다

        if goleo_link_cell.value and not bool(urlparse(goleo_link_cell.value).scheme):
            goleo_link_cell.value = None

        if naver_link_cell.value and not bool(urlparse(naver_link_cell.value).scheme):
            naver_link_cell.value = None

        # 가격차이 체크
        goleo_difference_cell = row[12]  # "고려 가격차이"는 13번째 컬럼에 위치
        naver_difference_cell = row[17]  # "네이버 가격차이"는 18번째 열입니다.

        if goleo_difference_cell.value is not None and goleo_difference_cell.value >= 0:
            for i in [10, 11, 12, 13, 14, 22]:
                row[i].value = None

        if naver_difference_cell.value is not None and naver_difference_cell.value >= 0:
            for i in [15, 16, 17, 18, 19, 20, 23]:
                row[i].value = None

        # '고려 가격차이(%)' 체크
        goleo_difference_percentage_cell = row[13]  # '고려 가격차이(%)'는 14번째 열입니다.
        if goleo_difference_percentage_cell.value is not None and goleo_difference_percentage_cell.value > -1:
            for i in [10, 11, 12, 13, 14, 22]:
                row[i].value = None

        # '네이버 가격차이(%)' 체크
        naver_difference_percentage_cell = row[18]  # '네이버 가격차이(%)'는 19번째 열입니다.
        if naver_difference_percentage_cell.value is not None and naver_difference_percentage_cell.value > -1:
            for i in [15, 16, 17, 18, 19, 20, 23]:
                row[i].value = None

        # '네이버 기본수량' 체크
        naver_quantity_cell = row[15]  # '네이버 기본수량'은 16번째 열입니다.
        naver_difference_percentage_cell = row[18]  # '네이버 가격차이(%)'는 19번째 열입니다.
        naver_text_cell = row[19]  # '네이버 공급사명'은 20번째 열입니다.

        # 체크할 단어들의 리스트
        check_words = ['판촉', '기프트', '답례품', '기념품', '인쇄', '각인', '제작', '호갱', '몽키', '홍보']

        # 단어가 포함되어 있는지 체크하는 함수
        def contains_keywords(text, keywords):
            for keyword in keywords:
                if keyword in text:
                    return True
            return False

        if (not naver_quantity_cell.value or str(naver_quantity_cell.value).strip() == "") and (naver_difference_percentage_cell.value is not None and naver_difference_percentage_cell.value > -10.0) and (not contains_keywords(str(naver_text_cell.value), check_words)):
            for i in [15, 16, 17, 18, 19, 20, 23]:
                row[i].value = None

        # '고려 기본수량' 체크
        goleo_quantity_cell = row[10]  # '고려 기본수량'은 11번째 컬럼에 위치
        if not (isinstance(goleo_quantity_cell.value, (int, float))):
            goleo_quantity_cell.value = None

    # 특정 컬럼들의 값이 모두 비어 있는지 확인하고, 그럴 경우 해당 행을 삭제
    for row in reversed(list(ws.iter_rows(min_row=2, max_row=ws.max_row))):
        columns_to_check = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]  # 위에서 제시된 컬럼 인덱스들

        # 해당 행의 컬럼 값들이 모두 None이거나 빈 문자열인지 확인
        if all([(cell.value is None or str(cell.value).strip() == "") for cell in [row[i] for i in columns_to_check]]):
            ws.delete_rows(row[0].row)
    
    # # '고려 가격차이(%)' 컬럼 삭제
    # col_num_to_delete = [col[0].column for col in ws.iter_cols() if col[0].value == '고려 가격차이(%)'][0]
    # ws.delete_cols(col_num_to_delete)
    
    wb.save(output_path)


In [2]:
filter_excel_file()

In [15]:
import os
from natsort import natsorted

def move_first_xlsx(source_dir, target_dir):
    files = [f for f in os.listdir(source_dir) if f.endswith('.xlsx')]
    files = natsorted(files)  # 파일 이름으로 정렬합니다.
    print(files)
    # if files:
    #     source_path = os.path.join(source_dir, files[0])  # 이동할 파일의 전체 경로
    #     target_path = os.path.join(target_dir, files[0])  # 대상 경로
        
    #     shutil.move(source_path, target_path)  # 파일 이동
    #     print(f"'{files[0]}' 파일을 '{source_dir}'에서 '{target_dir}'로 이동했습니다.")
    #     return True
    # else:
    #     print(f"'{source_dir}'에 .xlsx 파일이 없습니다.")
    #     return False

In [16]:
source_dir = r"C:\RPA\Input\Queue"
target_dir = r"C:\RPA\Input"

In [17]:
move_first_xlsx(source_dir, target_dir)

['6.볼펜10(300개)-가격관리-20231027 RPA2.xlsx', '7.볼펜11(300개)-가격관리-20231027 RPA2.xlsx', '8.볼펜12(300개)-가격관리-20231027 RPA2.xlsx', '9.볼펜13(300개)-가격관리-20231027 RPA2.xlsx', '10.볼펜15(300개)-가격관리-20231027 RPA2.xlsx', '11.송월6(300개)-가격관리-20231030 RPA2.xlsx', '12.송월7(300개)-가격관리-20231030 RPA2.xlsx', '13.송월8(300개)-가격관리-20231030 RPA2.xlsx', '14.송월9(194개)-가격관리-20231030 RPA2.xlsx', '15.USB4(300개)-가격관리-20231031.xlsx', '16.USB5(300개)-가격관리-20231031.xlsx', '17.USB6(158개)-가격관리-20231031.xlsx']


In [1]:
import os
import time
import openpyxl
import pandas as pd
import shutil
import psutil
import pyautogui
import pygetwindow as gw
from plyer import notification
from natsort import natsorted


# rpa 실행
def rpa_run(list_image_path, rpa_process_image_path, rpa_run_image_path, program_title, confidence_value=0.8):
    # 프로그램 창의 위치와 크기를 얻습니다.
    windows = gw.getWindowsWithTitle(program_title)
    if windows:
        window = windows[0]
        region = (window.left, window.top, window.width, window.height)
        
        # 이동한 후 클릭하려는 이미지 위치를 찾습니다.
        click_location = pyautogui.locateOnScreen(list_image_path, region=region, confidence=confidence_value)
        if click_location:
            click_center = pyautogui.center(click_location)
            pyautogui.click(click_center)  # rpa_list 이미지의 중앙 위치를 클릭합니다.
            
            # 클릭한 후 일정 시간 동안 대기합니다.
            time.sleep(2)
            
            # 대기 시간 후 rpa_process 이미지의 위치를 찾아서 마우스를 이동합니다.
            hover_location = pyautogui.locateOnScreen(rpa_process_image_path, region=region, confidence=confidence_value)
            if hover_location:
                hover_center = pyautogui.center(hover_location)  
                pyautogui.moveTo(hover_center)  # rpa_process 이미지의 중앙 위치로 마우스를 이동합니다.

                # 클릭한 후 일정 시간 동안 대기합니다.
                time.sleep(2)
                
                # 마우스 이동이 끝난 후 rpa_run 이미지를 클릭합니다.
                run_location = pyautogui.locateOnScreen(rpa_run_image_path, region=region, confidence=confidence_value)
                if run_location:
                    run_center = pyautogui.center(run_location)
                    pyautogui.click(run_center)  # run 이미지의 중앙 위치를 클릭합니다.
                else:
                    print(f"{program_title} 프로그램 내에서 {rpa_run_image_path} 이미지를 찾을 수 없습니다!")
            else:
                print(f"{program_title} 프로그램 내에서 {rpa_process_image_path} 이미지 클릭 후에 찾을 수 없습니다!")
        else:
            print(f"{program_title} 프로그램 내에서 {list_image_path} 이미지를 찾을 수 없습니다!")
    else:
        print(f"{program_title} 프로그램 창을 찾을 수 없습니다!")

def rpa_restart():
    source_dir = r"C:\RPA\Input\Queue"
    target_dir = r"C:\RPA\Input"
    config_workbook_path = 'C:\\RPA\\UserConfig.xlsx'
    process_name = "aworks_bot"
    list_image_path = r"C:\RPA\Image\Restart\rpa_list.jpg"
    rpa_process_image_path = r"C:\RPA\Image\Restart\rpa_process.jpg"
    rpa_run_image_path = r"C:\RPA\Image\Restart\rpa_run.jpg"
    program_title = "aworks_mini" # 확인하고 싶은 프로세스의 이름

    rpa_run(list_image_path, rpa_process_image_path, rpa_run_image_path, program_title)


In [2]:
rpa_restart()

In [6]:
from hyperlink_excel_file import hyperlink_excel_file

hyperlink_excel_file()

데이터가 C:\RPA\Output\조정건(234개)-20231212_result_20231213_132148.xlsx에 저장되었습니다.


In [5]:
import pandas as pd
import os
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Alignment, Border, Side
from urllib.parse import urlparse
from datetime import datetime

def filter_excel_file():
    input_directory = 'C:\\RPA\\Input'
    target_directory = 'C:\\RPA\\Output'
    
    xlsx_files = [f for f in os.listdir(input_directory) if f.lower().endswith('.xlsx')]
    
    if not xlsx_files:
        print("xlsx 파일이 없습니다.")
        return
    
    input_filename = xlsx_files[0].rsplit('.', 1)[0] # 확장자 제외한 파일명
    current_datetime = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_filename = f"{input_filename}_upload_{current_datetime}.xlsx"

    file_path = os.path.join(input_directory, xlsx_files[0])
    df = pd.read_excel(file_path)
    
    filtered_df = df[(df['가격차이(2)'] < 0) | (df['가격차이(3)'] < 0)]

    # 원하는 컬럼만 추출
    filtered_df = filtered_df[['구분', '담당자', '업체명', '업체코드', 'Code', '중분류카테고리', '상품명', '기본수량(1)', '판매단가(V포함)', '본사상품링크', '기본수량(2)', '판매단가(V포함)(2)', '가격차이(2)', '가격차이(2)(%)', '고려기프트 상품링크', '기본수량(3)', '판매단가(V포함)(3)', '가격차이(3)', '가격차이(3)(%)', '공급사명', '공급사 상품링크', '본사 이미지', '고려기프트 이미지', '네이버 이미지']]

    # 컬럼 이름 변경
    filtered_df.rename(columns={
        '구분': '구분(승인관리:A/가격관리:P)',
        '담당자': '담당자',
        '업체명': '공급사명',
        '업체코드': '공급처코드',
        'Code': '상품코드',
        '중분류카테고리': '카테고리(중분류)',
        '상품명': '상품명',
        '기본수량(1)': '본사 기본수량',
        '판매단가(V포함)': '판매단가1(VAT포함)',
        '본사상품링크': '본사링크',
        '기본수량(2)': '고려 기본수량',
        '판매단가(V포함)(2)': '판매단가2(VAT포함)',
        '가격차이(2)': '고려 가격차이',
        '가격차이(2)(%)': '고려 가격차이(%)',
        '고려기프트 상품링크': '고려 링크',
        '기본수량(3)': '네이버 기본수량',
        '판매단가(V포함)(3)': '판매단가3 (VAT포함)',
        '가격차이(3)': '네이버 가격차이',
        '가격차이(3)(%)': '네이버가격차이(%)',
        '공급사명': '네이버 공급사명',
        '공급사 상품링크': '네이버 링크',
        '본사 이미지': '해오름(이미지링크)',
        '고려기프트 이미지': '고려기프트(이미지링크)',
        '네이버 이미지': '네이버쇼핑(이미지링크)'
    }, inplace=True)

    output_path = os.path.join(target_directory, output_filename)
    filtered_df.to_excel(output_path, index=False)
    
    # 여기서부터 openpyxl을 사용해서 추가 작업
    wb = load_workbook(output_path)
    ws = wb.active
    
    # 스타일 설정
    thin_border = Border(left=Side(style='thin'), right=Side(style='thin'),
                         top=Side(style='thin'), bottom=Side(style='thin'))

    for row in ws.iter_rows(min_row=2, max_col=24, max_row=ws.max_row):
        for cell in row:
            cell.border = thin_border

    for col in ws.iter_cols(min_col=1, max_col=24):
        for cell in col:
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    
    # 너비와 높이 설정
    for col in ws.iter_cols(min_col=1, max_col=24, max_row=1):
        for cell in col:
            ws.column_dimensions[cell.column_letter].width = 5 # 15
            
    for row in ws.iter_rows(min_row=1, max_col=24, max_row=ws.max_row):
        for cell in row:
            ws.row_dimensions[cell.row].height = 16.5 # 100

    columns_to_check = [col[0].column for col in ws.iter_cols() if col[0].value in ['고려 기본수량', '판매단가2(VAT포함)', '고려 가격차이', '고려 가격차이(%)', '고려 링크', '네이버 기본수량', '판매단가3 (VAT포함)', '네이버 가격차이', '네이버가격차이(%)', '네이버 공급사명', '네이버 링크', '해오름(이미지링크)', '고려기프트(이미지링크)', '네이버쇼핑(이미지링크)']]
    
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
        for col_num in columns_to_check:
            cell = row[col_num-1]
            if cell.value == '-':
                cell.value = ''

    for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
        # "고려 링크"와 "네이버 링크" 체크
        goleo_link_cell = row[14]  # "고려 링크"는 15번째 열입니다.
        naver_link_cell = row[20]  # "네이버 링크"는 21번째 열입니다

        if goleo_link_cell.value and not bool(urlparse(goleo_link_cell.value).scheme):
            goleo_link_cell.value = None

        if naver_link_cell.value and not bool(urlparse(naver_link_cell.value).scheme):
            naver_link_cell.value = None

        # 가격차이 체크
        goleo_difference_cell = row[12]  # "고려 가격차이"는 13번째 컬럼에 위치
        naver_difference_cell = row[17]  # "네이버 가격차이"는 18번째 열입니다.

        if goleo_difference_cell.value is not None and goleo_difference_cell.value >= 0:
            for i in [10, 11, 12, 13, 14, 22]:
                row[i].value = None

        if naver_difference_cell.value is not None and naver_difference_cell.value >= 0:
            for i in [15, 16, 17, 18, 19, 20, 23]:
                row[i].value = None

        # '고려 가격차이(%)' 체크
        for row_index, row in enumerate(ws.iter_rows(min_row=2, max_row=ws.max_row), start=2):
            goleo_difference_percentage_cell = row[13]  # '고려 가격차이(%)'는 14번째 열입니다.
            try:
                goleo_difference_percentage = float(goleo_difference_percentage_cell.value)
                if goleo_difference_percentage > -1:
                    for i in [10, 11, 12, 13, 14, 22]:
                        row[i].value = None
            except ValueError:
                print(f"에러가 발생한 행: {row_index}, 셀 값: {goleo_difference_percentage_cell.value}")

        # '네이버 가격차이(%)' 체크
        for row_index, row in enumerate(ws.iter_rows(min_row=2, max_row=ws.max_row), start=2):
            naver_difference_percentage_cell = row[18]  # '네이버 가격차이(%)'는 19번째 열입니다.
            try:
                naver_difference_percentage = float(naver_difference_percentage_cell.value)
                if naver_difference_percentage > -1:
                    for i in [15, 16, 17, 18, 19, 20, 23]:
                        row[i].value = None
            except ValueError:
                print(f"에러가 발생한 행: {row_index}, 셀 값: {naver_difference_percentage_cell.value}")

        # '네이버 기본수량' 체크
        naver_quantity_cell = row[15]  # '네이버 기본수량'은 16번째 열입니다.
        naver_difference_percentage_cell = row[18]  # '네이버 가격차이(%)'는 19번째 열입니다.
        naver_text_cell = row[19]  # '네이버 공급사명'은 20번째 열입니다.

        # 체크할 단어들의 리스트
        check_words = ['판촉', '기프트', '답례품', '기념품', '인쇄', '각인', '제작', '호갱', '몽키', '홍보']

        # 단어가 포함되어 있는지 체크하는 함수
        def contains_keywords(text, keywords):
            for keyword in keywords:
                if keyword in text:
                    return True
            return False

        if (not naver_quantity_cell.value or str(naver_quantity_cell.value).strip() == "") and (naver_difference_percentage_cell.value is not None and naver_difference_percentage_cell.value > -10.0) and (not contains_keywords(str(naver_text_cell.value), check_words)):
            for i in [15, 16, 17, 18, 19, 20, 23]:
                row[i].value = None

        # '고려 기본수량' 체크
        goleo_quantity_cell = row[10]  # '고려 기본수량'은 11번째 컬럼에 위치
        if not (isinstance(goleo_quantity_cell.value, (int, float))):
            goleo_quantity_cell.value = None

    # 특정 컬럼들의 값이 모두 비어 있는지 확인하고, 그럴 경우 해당 행을 삭제
    for row in reversed(list(ws.iter_rows(min_row=2, max_row=ws.max_row))):
        columns_to_check = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]  # 위에서 제시된 컬럼 인덱스들

        # 해당 행의 컬럼 값들이 모두 None이거나 빈 문자열인지 확인
        if all([(cell.value is None or str(cell.value).strip() == "") for cell in [row[i] for i in columns_to_check]]):
            ws.delete_rows(row[0].row)
    
    # # '고려 가격차이(%)' 컬럼 삭제
    # col_num_to_delete = [col[0].column for col in ws.iter_cols() if col[0].value == '고려 가격차이(%)'][0]
    # ws.delete_cols(col_num_to_delete)
    
    wb.save(output_path)


TypeError: '>' not supported between instances of 'str' and 'int'